# 00 – Data setup (Collect and clean SPY 1-minute data)

This notebook implements the "Data Sources" section:

- Load raw SPY 1-minute OHLCV+volume data from `data/raw/`.
- Convert Unix timestamps to U.S. Eastern Time.
- Keep only the regular session (09:30–16:00 ET).
- Restrict to about 55 trading days starting 8 September 2025.
- Drop the per-minute open and keep only high, low, close, and volume.
- Save a cleaned dataset into `data/clean/` for all later notebooks.


In [ ]:
#looking at the data to see if its in correct form

from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path("..").resolve()

DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_CLEAN = PROJECT_ROOT / "data" / "clean"

RAW_FILE = DATA_RAW / "spy_1min_bats_2025.csv"

print("RAW_FILE path:", RAW_FILE)
print("File exists?:", RAW_FILE.exists())


RAW_FILE path: /Users/canka/Dev/python/DSA210-Project-Can-Karadogan/data/raw/spy_1min_bats_2025.csv
File exists?: True
